# Magnetostatics

In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
import math

### model of the coil:

In [2]:
cyl = Cylinder((0,0,0), Z, r=0.01, h=0.03).faces[0]
heli = Edge(Segment((0,0), (12*math.pi, 0.03)), cyl)
ps = heli.start
vs = heli.start_tangent
pe = heli.end
ve = heli.end_tangent

e1 = Segment((0,0,-0.03), (0,0,-0.01))
c1 = BezierCurve( [(0,0,-0.01), (0,0,0), ps-vs, ps])
e2 = Segment((0,0,0.04), (0,0,0.06))
c2 = BezierCurve( [pe, pe+ve, (0,0,0.03), (0,0,0.04)])
spiral = Wire([e1, c1, heli, c2, e2])
circ = Face(Wire([Circle((0,0,-0.03), Z, 0.001)]))
coil = Pipe(spiral, circ)

coil.faces.maxh=0.2
coil.faces.name="coilbnd"
coil.faces.Max(Z).name="in"
coil.faces.Min(Z).name="out"
coil.mat("coil")
crosssection = coil.faces.Max(Z).mass

In [3]:
DrawGeo (coil);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [4]:
box = Box((-0.04,-0.04,-0.03), (0.04,0.04,0.06))
box.faces.name = "outer"
air = box-coil
air.mat("air");

### mesh-generation of coil and air-box:

In [5]:
geo = OCCGeometry(Glue([coil,air]))
with TaskManager():
    mesh = Mesh(geo.GenerateMesh(meshsize.coarse, maxh=0.01)).Curve(3)
    
Draw (mesh, clipping={"y":1, "z":0, "dist":0.012});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

checking mesh data materials and boundaries:

In [6]:
mesh.ne, mesh.nv, mesh.GetMaterials(), mesh.GetBoundaries()

(155334,
 26775,
 ('coil', 'air'),
 ('out',
  'coilbnd',
  'coilbnd',
  'coilbnd',
  'coilbnd',
  'coilbnd',
  'in',
  'outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'outer'))

### Solve a potential problem to determine current density in wire:
on the domain $\Omega_{\text{coil}}$:
\begin{eqnarray*}
j & = & \sigma \nabla \Phi \\
\operatorname{div} j & = & 0
\end{eqnarray*}
port boundary conditions: 
\begin{eqnarray*}
\Phi & = & 0  \qquad \qquad \text{on } \Gamma_{\text{out}},  \\
j_n & = & \frac{1}{|S|} \quad \qquad \text{on } \Gamma_{\text{in}},
\end{eqnarray*}
and $j_n=0$ else

In [7]:
fespot = H1(mesh, order=3, definedon="coil", dirichlet="out")
phi,psi = fespot.TnT()
sigma = 58.7e6
bfa = BilinearForm(sigma*grad(phi)*grad(psi)*dx).Assemble()
inv = bfa.mat.Inverse(freedofs=fespot.FreeDofs(), inverse="sparsecholesky")
lff = LinearForm(1/crosssection*psi*ds("in")).Assemble()
gfphi = GridFunction(fespot)
gfphi.vec.data = inv * lff.vec

In [8]:
Draw (gfphi, draw_vol=False, clipping={"y":1, "z":0, "dist":0.012});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

### Solve magnetostatic problem:

current source is current from potential equation:

$$
\int \mu^{-1} \operatorname{curl} u \cdot \operatorname{curl} v \, dx =
\int j \cdot v \, dx
$$


In [9]:
fes = HCurl(mesh, order=2, nograds=True)
print ("HCurl dofs:", fes.ndof)
u,v = fes.TnT()
mu = 4*math.pi*1e-7
a = BilinearForm(1/mu*curl(u)*curl(v)*dx+1e-6/mu*u*v*dx)
pre = Preconditioner(a, "bddc")
f = LinearForm(sigma*grad(gfphi)*v*dx("coil"))
with TaskManager():
    a.Assemble()
    f.Assemble()

HCurl dofs: 805484


In [10]:
from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates=True)
gfu = GridFunction(fes)
with TaskManager():
    gfu.vec.data = inv * f.vec

CG iteration 1, residual = 23.408072277907568     
CG iteration 2, residual = 0.079662967180086     
CG iteration 3, residual = 0.018919386212224474     
CG iteration 4, residual = 0.013578604785837159     
CG iteration 5, residual = 0.008003446326698804     
CG iteration 6, residual = 0.005160055030359998     
CG iteration 7, residual = 0.0032986144389692868     
CG iteration 8, residual = 0.0023102657097962866     
CG iteration 9, residual = 0.0016030816920312942     
CG iteration 10, residual = 0.001193425021950073     
CG iteration 11, residual = 0.0009315010323604369     
CG iteration 12, residual = 0.0007351004081146959     
CG iteration 13, residual = 0.0006179325196965421     
CG iteration 14, residual = 0.0005083966602214831     
CG iteration 15, residual = 0.0004271640599436244     
CG iteration 16, residual = 0.00034168716874909686     
CG iteration 17, residual = 0.00026716950208279693     
CG iteration 18, residual = 0.00019413567830834386     
CG iteration 19, residual = 

In [11]:
Draw (curl(gfu), mesh, draw_surf=False, \
      min=0, max=3e-4, clipping = { "y":1, "z" : 0, "function":False}, vectors = { "grid_size":100});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…